In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pymatgen.io.cif import CifParser
from pymatgen.ext.matproj import MPRester

In [2]:
mp_key = "NKfarLsNHvBTHM4FxxhUlnMjaXDHo7fy"
mp_id = "mp-22473"

In [3]:
with MPRester(mp_key) as m:
    primitive_structure = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conventionnelle_structure = SpacegroupAnalyzer(primitive_structure).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conventionnelle_structure).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Mon travail

**_____________________________________________________________________________**
## Vecteurs de base des réseaux directs et indirects
**_____________________________________________________________________________**

In [4]:
# Vecteurs de base du réseau direct de la maille primitive
ap, bp, cp = primitive_structure.lattice.matrix
print(f"Les vecteurs de base du réseau direct de la maille primitive sont:\n\ta = {np.round(ap,8)}\n\tb = {np.round(bp,8)}\n\tc = {np.round(cp,8)}")

# Vecteurs de base du réseau réciproque de la maille primitive
arp, brp, crp = primitive_structure.lattice.reciprocal_lattice.matrix
print(f"\n\nLes vecteurs de base du réseau réciproque de la maille primitive sont:\n\ta = {np.round(arp,8)}\n\tb = {np.round(brp,8)}\n\tc = {np.round(crp,8)}")

# Vecteurs de base du réseau direct de la maille conventionnelle
ac, bc, cc = conventionnelle_structure.lattice.matrix
print(f"\n\n\n\n\nLes vecteurs de base du réseau direct de la maille conventionnelle sont:\n\ta = {np.round(ac,8)}\n\tb = {np.round(bc,8)}\n\tc = {np.round(cc,8)}")

# Vecteurs de base du réseau réciproque de la maille conventionnelle
arc, brc, crc = conventionnelle_structure.lattice.reciprocal_lattice.matrix
print(f"\n\nLes vecteurs de base du réseau réciproque de la maille conventionnelle sont:\n\ta = {np.round(arc,8)}\n\tb = {np.round(brc,8)}\n\tc = {np.round(crc,8)}")


Les vecteurs de base du réseau direct de la maille primitive sont:
	a = [ 3.83291715 -0.00880891  6.24287955]
	b = [1.75686193 3.40657585 6.24287956]
	c = [-0.0144931  -0.00880892  7.32561187]


Les vecteurs de base du réseau réciproque de la maille primitive sont:
	a = [ 1.63371781 -0.84660942  0.00221414]
	b = [6.23020e-04 1.84005e+00 2.21386e-03]
	c = [-1.39278371 -0.84660913  0.85392756]





Les vecteurs de base du réseau direct de la maille conventionnelle sont:
	a = [ 1.99842811 -3.46137903  0.        ]
	b = [1.99842811 3.46137903 0.        ]
	c = [ 0.          0.         20.85807446]


Les vecteurs de base du réseau réciproque de la maille conventionnelle sont:
	a = [ 1.57203186 -0.90761301  0.        ]
	b = [1.57203186 0.90761301 0.        ]
	c = [0.         0.         0.30123516]


## Commentaires sur les réseaux directs et réciproques

**Réseau direct:** Un réseau cristallin est défini par ses vecteurs de base a, b, et c indépendants et tels que tout point du réseau peut s’écrire:

            R = l a + m b + n c avec l, m, n ∈ ℤ
            
**Réseau réciproque:** On peut obtenir le réseau réciproque à partir des vecteurs de base. Le réseau réciproque est l'ensemble des vecteurs qui vérifient la condition de Laue.

**_____________________________________________________________________________**
## Type de maille, système cristallin et groupe ponctuel.
**_____________________________________________________________________________**

In [7]:
# Type de maille
analyseur = SpacegroupAnalyzer(primitive_structure)
type_maille = analyseur.get_space_group_symbol()[0]
print("Type de maille:", type_maille)

# Longueurs et angles des vecteurs de base du réseau direct de la structure primitive --> Structure cristalline
long_ac, long_bc, long_cc = np.linalg.norm(primitive_structure.lattice.matrix, axis = 1)
alpha, beta, gamma = primitive_structure.lattice.angles  # Angles en degrés

print(100*"_"+f"\nLongueur des vecteurs de base:\n\t||a|| = {round(long_ac,3)} [Å]\n\t||b|| = {round(long_bc,3)} [Å]\n\t||c|| = {round(long_cc,3)} [Å]")
print(f"\n\nAngles de base:\n\tα = {round(alpha,3)}°\n\tβ = {round(beta,3)}°\n\tγ = {round(gamma,3)}°")
print(f"\n\nDu fait que ||a|| = ||b|| = ||c|| et que α = β = γ != 90°, on en déduit que le système\ncritallin est trigonal (aussi appelé rhombohédrique.)")

# Groupe ponctuel
type_groupe_ponctuel = analyseur.get_space_group_symbol()[1:]
print(100*"_"+"\nType de groupe ponctuel:", type_groupe_ponctuel)
print("Le système poncutel est un système trigonal 3m.")
print("Le -3 représente une roto-inversion d'ordre 3 le long de la direction [111].")
print("Le m représente la réflexion à travers un plan miroir le long des directions [110], [011], et [101].")

Type de maille: R
____________________________________________________________________________________________________
Longueur des vecteurs de base:
	||a|| = 7.326 [Å]
	||b|| = 7.326 [Å]
	||c|| = 7.326 [Å]


Angles de base:
	α = 31.662°
	β = 31.662°
	γ = 31.662°


Du fait que ||a|| = ||b|| = ||c|| et que α = β = γ != 90°, on en déduit que le système
critallin est trigonal (aussi appelé rhombohédrique.)
____________________________________________________________________________________________________
Type de groupe ponctuel: -3m
Le système poncutel est un système trigonal 3m.
Le -3 représente une roto-inversion d'ordre 3 le long de la direction [111].
Le m représente la réflexion à travers un plan miroir le long des directions [110], [011], et [101].
